In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
cities_file = "Resources/cities.csv"
cities_df = pd.read_csv(cities_file)
cities_df.head()

,id,name,coords,start_year,url_name,country,country_state
0,5,Aberdeen,POINT(-2.15 57.15),2017.0,aberdeen,Scotland,NaN
1,6,Adelaide,POINT(138.6 -34.91666667),2017.0,adelaide,Australia,NaN
2,7,Algiers,POINT(3 36.83333333),2017.0,algiers,Algeria,NaN
3,9,Ankara,POINT(32.91666667 39.91666667),2017.0,ankara,Turkey,NaN
4,16,Belém,POINT(-48.48333333 -1.466666667),2017.0,belem,Brazil,NaN


In [4]:
stations_file = "Resources/stations.csv"
stations_df = pd.read_csv(stations_file)
stations_df.head()

,id,name,geometry,buildstart,opening,closure,city_id
0,7694,Keisei Tsudanuma,POINT(140.024812197129 35.6837744784723),1921.0,1921.0,999999.0,114
1,6003,Kossuth Lajos tér,POINT(19.0462376564033 47.5054880717671),0.0,0.0,999999.0,29
2,7732,Saint-Charles,POINT(5.3801556 43.3024646),1973.0,1977.0,999999.0,74
3,7695,Keisei Makuhari-Hongo,POINT(140.042146725175 35.6726021159981),1991.0,1991.0,999999.0,114
4,7726,Chartreux,POINT(5.4014815 43.309129),1973.0,1977.0,999999.0,74
